In [13]:
import os

import torch
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [14]:
torch.set_float32_matmul_precision("medium")

In [19]:
class TestNet(nn.Module):
    def __init__(self):
        super(TestNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [20]:
seed = 13
torch.manual_seed(seed)

use_accel = True
save_model = True
dry_run = False
log_interval = 50

batch_size = 6
test_batch_size = 1
epochs = 1

lr = 0.001
gamma = 0.99

if use_accel:
    device = torch.accelerator.current_accelerator()
else:
    device = torch.device("cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

if use_accel:
    accel_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(accel_kwargs)
    test_kwargs.update(accel_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

dataset1 = MNIST('../data/datasets/', train=True, download=True,
                   transform=transform)
dataset2 = MNIST('../data/datasets/', train=False,
                   transform=transform)

train_loader = DataLoader(dataset1,**train_kwargs)
test_loader = DataLoader(dataset2, **test_kwargs)

In [21]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

In [22]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [25]:
network = TestNet().to(device)
optimizer = optim.Adadelta(network.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [26]:
for epoch in range(1, epochs + 1):
    train(network, device, train_loader, optimizer, epoch)
    test(network, device, test_loader)
    scheduler.step()

if save_model:
    torch.save(network.state_dict(), "../checkpoints/MNIST/mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.325482
Train Epoch: 1 [300/60000 (0%)]	Loss: 2.314932
Train Epoch: 1 [600/60000 (1%)]	Loss: 2.196333
Train Epoch: 1 [900/60000 (2%)]	Loss: 2.243163
Train Epoch: 1 [1200/60000 (2%)]	Loss: 2.233226
Train Epoch: 1 [1500/60000 (2%)]	Loss: 2.219179
Train Epoch: 1 [1800/60000 (3%)]	Loss: 2.199193
Train Epoch: 1 [2100/60000 (4%)]	Loss: 2.286519
Train Epoch: 1 [2400/60000 (4%)]	Loss: 2.173641
Train Epoch: 1 [2700/60000 (4%)]	Loss: 2.243483
Train Epoch: 1 [3000/60000 (5%)]	Loss: 2.319979
Train Epoch: 1 [3300/60000 (6%)]	Loss: 2.281537
Train Epoch: 1 [3600/60000 (6%)]	Loss: 2.184864
Train Epoch: 1 [3900/60000 (6%)]	Loss: 2.228739
Train Epoch: 1 [4200/60000 (7%)]	Loss: 2.116406
Train Epoch: 1 [4500/60000 (8%)]	Loss: 2.161614
Train Epoch: 1 [4800/60000 (8%)]	Loss: 2.125653
Train Epoch: 1 [5100/60000 (8%)]	Loss: 2.126293
Train Epoch: 1 [5400/60000 (9%)]	Loss: 2.093744
Train Epoch: 1 [5700/60000 (10%)]	Loss: 2.178246
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.223